# Multiple dispatch

In this notebook we'll explore **multiple dispatch**, which is a key feature of Julia.

Multiple dispatch makes software *generic* and *fast*!

#### Starting with the familiar

To understand multiple dispatch in Julia, let's start with what we've already seen.

We can declare functions in Julia without giving Julia any information about the types of the input arguments that function will receive:

In [1]:
f(x) = x^2

f (generic function with 1 method)

and then Julia will determine on its own which input argument types make sense and which do not:

In [2]:
f(10)

100

_**Note:**_  
On the other hand, Julia does not know what to do with a vector when we try to square it, so it gives an error message...

In [3]:
f([1, 2, 3])

LoadError: MethodError: no method matching ^(::Vector{Int64}, ::Int64)
[0mClosest candidates are:
[0m  ^([91m::Union{AbstractChar, AbstractString}[39m, ::Integer) at strings/basic.jl:718
[0m  ^([91m::Complex{var"#s79"} where var"#s79"<:AbstractFloat[39m, ::Integer) at complex.jl:818
[0m  ^([91m::Complex{var"#s79"} where var"#s79"<:Integer[39m, ::Integer) at complex.jl:820
[0m  ...

#### Specifying the types of our input arguments

However, we also have the *option* to tell Julia explicitly what types our input arguments are allowed to have.

For example, let's write a function `foo` that only takes strings as inputs.

In [4]:
foo(x::String, y::String) = println("My inputs x and y are both strings!")

foo (generic function with 1 method)

We see here that in order to restrict the type of `x` and `y` to `String`s, we just follow the input argument name by a double colon and the keyword `String`.

Now we'll see that `foo` works on `String`s and doesn't work on other input argument types.

In [5]:
foo("hello", "hi!")

My inputs x and y are both strings!


_**Note:**_   
foo on 3 & 4, we get a error, and the reason is that we don't have a function called foo that works on integers

In [6]:
foo(3, 4)

LoadError: MethodError: no method matching foo(::Int64, ::Int64)

To get `foo` to work on integer (`Int`) inputs, let's tack `::Int` onto our input arguments when we declare `foo`.

In [7]:
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")

foo (generic function with 2 methods)

In [8]:
foo(3, 4)

My inputs x and y are both integers!


Now `foo` works on integers! But look, `foo` also still works when `x` and `y` are strings!

In [9]:
foo("hello", "hi!")

My inputs x and y are both strings!


This is starting to get to the heart of multiple dispatch. When we declared

```julia
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")
```
we didn't overwrite or replace
```julia
foo(y::String, y::String)```

Instead, we just added an additional ***method*** to the ***generic function*** called `foo`.

A ***generic function*** is the abstract concept associated with a particular operation.

For example, the generic function `+` represents the concept of addition.

A ***method*** is a specific implementation of a generic function for *particular argument types*.

For example, `+` has methods that accept floating point numbers, integers, matrices, etc.

We can use the `methods` to see how many methods there are for `foo`.

In [10]:
methods(foo)

# 2 methods for generic function "foo":
[1] foo(x::String, y::String) in Main at In[4]:1
[2] foo(x::Int64, y::Int64) in Main at In[7]:1

Aside: how many methods do you think there are for addition?

In [11]:
methods(+)

# 190 methods for generic function "+":
[1] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87
[2] +(c::Union{UInt16, UInt32, UInt8}, x::BigInt) in Base.GMP at gmp.jl:528
[3] +(c::Union{Int16, Int32, Int8}, x::BigInt) in Base.GMP at gmp.jl:534
[4] +(c::Union{UInt16, UInt32, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:376
[5] +(c::Union{Int16, Int32, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:384
[6] +(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:392
[7] +(x::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\Dates\src\periods.jl:362
[8] +(A::LinearAlgebra.UpperHessenberg, B::LinearAlgebra.UpperHessenberg) in LinearAlgebra at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\LinearAlgebra\src\hessenberg.jl:101
[9] +(H::LinearAlgebra.UpperHessenberg, J::LinearAlgebra.UniformScaling) in LinearAlgebra at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\LinearAlgebra\src\hessenberg.jl:106
[10] +(x::AbstractIrrational, y::AbstractIrrational) in Base at irrationals.jl:156
[11] +(x::T, y::Integer) where T<:AbstractChar in Base at char.jl:223
[12] +(x::P, y::P) where P<:Dates.Period in Dates at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\Dates\src\periods.jl:79
[13] +(x::Dates.Period, y::Dates.Period) in Dates at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\Dates\src\periods.jl:351
[14] +(y::Dates.Period, x::Dates.CompoundPeriod) in Dates at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\Dates\src\periods.jl:353
[15] +(y::Dates.Period, x::Dates.TimeType) in Dates at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\Dates\src\arithmetic.jl:85
[16] +(x::Dates.Period, r::AbstractRange{var"#s832"} where var"#s832"<:Dates.TimeType) in Dates at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\Dates\src\ranges.jl:62
[17] +(y::Union{Dates.CompoundPeriod, Dates.Period}, x::AbstractArray{var"#s832", N} where {var"#s832"<:Dates.TimeType, N}) in Dates at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\Dates\src\deprecated.jl:14
[18] +(A::LinearAlgebra.Bidiagonal, B::LinearAlgebra.Bidiagonal) in LinearAlgebra at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\LinearAlgebra\src\bidiag.jl:354
[19] +(A::LinearAlgebra.Bidiagonal, B::LinearAlgebra.UpperTriangular) in LinearAlgebra at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\LinearAlgebra\src\special.jl:94
[20] +(A::LinearAlgebra.Bidiagonal, B::LinearAlgebra.UnitUpperTriangular) in LinearAlgebra at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\LinearAlgebra\src\special.jl:94
[21] +(A::LinearAlgebra.Bidiagonal, B::LinearAlgebra.LowerTriangular) in LinearAlgebra at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\LinearAlgebra\src\special.jl:94
[22] +(A::LinearAlgebra.Bidiagonal, B::LinearAlgebra.UnitLowerTriangular) in LinearAlgebra at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\LinearAlgebra\src\special.jl:94
[23] +(A::LinearAlgebra.Bidiagonal, B::LinearAlgebra.Diagonal) in LinearAlgebra at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\LinearAlgebra\src\special.jl:115
[24] +(A::LinearAlgebra.Bidiagonal, B::LinearAlgebra.Tridiagonal) in LinearAlgebra at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\LinearAlgebra\src\special.jl:183
[25] +(A::LinearAlgebra.Bidiagonal, B::LinearAlgebra.SymTridiagonal) in LinearAlgebra at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\LinearAlgebra\src\special.jl:203
[26] +(A::LinearAlgebra.Bidiagonal{var"#s832", V} where {var"#s832"<:Number, V<:AbstractVector{var"#s832"}}, B::LinearAlgebra.UniformScaling) in LinearAlgebra at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\LinearAlgebra\src\special.jl:236
[27] +(y::AbstractFloat, x::Bool) in Base at bool.jl:102
[28] +(A::LinearAlgebra.UpperTriangular, B::LinearAlgebra.UpperTriangular) in LinearAlgebra at D:\Programs\Julia-1.6.2\share\julia\stdlib\v1.6\LinearAlgebra\src\triangular.jl:648
[29] +(A::LinearAlgebra.UpperTriangular, B::LinearAlgebra.UnitUpper

So, we now can call `foo` on integers or strings. When you call `foo` on a particular set of arguments, Julia will infer the types of the inputs and dispatch the appropriate method. *This* is multiple dispatch.

Multiple dispatch makes our code generic and fast. Our code can be generic and flexible because we can write code in terms of abstract operations such as addition and multiplication, rather than in terms of specific implementations. At the same time, our code runs quickly because Julia is able to call efficient methods for the relevant types.

To see which method is being dispatched when we call a generic function, we can use the @which macro:

In [12]:
@which foo(3, 4)

foo(x::Int64, y::Int64) in Main at In[7]:1

Let's see what happens when we use `@which` with the addition operator!

In [13]:
@which 3.0 + 3.0

+(x::Float64, y::Float64) in Base at float.jl:326

And we can continue to add other methods to our generic function `foo`. Let's add one that takes the ***abstract type*** `Number`, which includes subtypes such as `Int`, `Float64`, and other objects you would think of as numbers:

In [14]:
foo(x::Number, y::Number) = println("My inputs x and y are both numbers!")

foo (generic function with 3 methods)

This method for `foo` will work on, for example, floating point numbers:

In [15]:
foo(3.0, 4.0)

My inputs x and y are both numbers!


We can also add a fallback, duck-typed method for `foo` that takes inputs of any type:

In [16]:
foo(x, y) = println("I accept inputs of any type!")

foo (generic function with 4 methods)

Given the methods we've already written for `foo` so far, this method will be called whenever we pass non-numbers to `foo`:

In [17]:
v = rand(3)
foo(v, v)

I accept inputs of any type!


### Exercises

#### 9.1

Extend the function `foo`, adding a method that takes only one input argument, which is of type `Bool`, and prints "foo with one boolean!"

#### 9.2

Check that the method being dispatched when you execute 
```julia
foo(true)
```
is the one you wrote.